In [17]:
from IPython.core.display import display

%load_ext tensorboard

from datetime import datetime
from models import GeneratorForMnistGLO as Gen
import datasets
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import pickle
def save_pickle(path,object):
    with open(path, 'wb') as handle:
        pickle.dump(object, handle, protocol=pickle.HIGHEST_PROTOCOL)


import matplotlib.pyplot  as plt

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [10]:
IMAGES = 0
CONTENT_LABEL = 2
CLASS_LABEL = 1
LR="ADAM_DEFULAT"
EPOCHS=300
noise_std=5
BATCH_SIZE=32
WD=0
CLASS_DIM = 5
CONTENT_DIM = 25
TOTAL_VEC_DIM = CLASS_DIM+CONTENT_DIM


ToDo
- Each style : Thick , rotated , thin
- Does an overfit model can detect style?
- Flow chart of distances to numbers.
- Maybe clusters
- Comapre weight decay  and L1,L2 Loss
- Compare differnet wieght decay values : 0 , 0.01,0.1,0.5 and show the results.
- THAT'S IT , NO MORE YOU HAVE TO MOVE ON.

- If time allow : compare different dimensions for vector dim , compare different noises

In [11]:
mnist_d=datasets.MNIST(2000)
digits=np.zeros((2000,28,28))
for i,data in enumerate(mnist_d):
    digits[i,:,:]=data[0]
digits=torch.tensor(digits)
print(digits.shape)




100%|██████████| 10000/10000 [00:00<00:00, 27405.58it/s]


torch.Size([2000, 28, 28])


In [12]:
class GLO(nn.Module):
    """

    """
    def __init__(self, code_dim, out_channels=1):
        super(GLO, self).__init__()
        self.embedding_content=nn.Embedding(2000,CONTENT_DIM)
        self.embedding_class=nn.Embedding(10,CLASS_DIM)
        self.generator=Gen(code_dim,out_channels=out_channels)

    def forward(self, class_id,image_id):
        curr_class_embedding=self.embedding_class(class_id)
        curr_content_embedding=self.embedding_content(image_id)
        if self.training:
            curr_content_embedding+=torch.normal(0.0,noise_std,size=curr_content_embedding.shape)
            
        combined=torch.cat((curr_content_embedding,curr_class_embedding),dim=1)
        generated_pic=self.generator(combined)
        generated_pic=torch.squeeze(generated_pic,dim=1)
        return generated_pic

    def get_emb_class(self,num):
        return self.embedding_class(num)
    def get_emb_content(self,id):
        return self.embedding_content(id)

    def save_weights(self,name):
        return save_pickle(name,self.state_dict())


In [13]:
cur_model=GLO(TOTAL_VEC_DIM)
print(3)

3


In [14]:
# image=np.zeros((2000,1,28,28))
# ids=np.zeros(2000)
# classes=np.zeros(2000)
# a=np.arange(10)
# import torchvision.models as models

# IMG_SIZE=224
# num_pics=np.zeros((10,1,28,28))
# for i,pic in enumerate(mnist_d):
#     ids[i]=pic[2]
#     curr_num = pic[1]
#     classes[i]=curr_num
#     image[i,:,:,:]=pic[0]

#     if curr_num in a:
#         num_pics[curr_num,:,:,:]=pic[0]
#         a[curr_num]=-1
        
# from collections import namedtuple
# import torchvision.models as models

# import cv2

# def resize(img_array):
#     tmp = np.empty((img_array.shape[0],3, IMG_SIZE, IMG_SIZE))
#     for i in range(len(img_array)):
#         img = img_array[i].reshape(28, 28).astype('uint8')
#         img = cv2.resize(img_array[i], (IMG_SIZE, IMG_SIZE))
#         img = img.astype('float32')/255
#         tmp[i,0,:,:] = img
#         tmp[i,1,:,:] = img
#         tmp[i,2,:,:] = img

        
#     return tmp

LossOutput = namedtuple("LossOutput", ["relu1_2", "relu2_2", "relu3_3", "relu4_3"])
# https://discuss.pytorch.org/t/how-to-extract-features-of-an-image-from-a-trained-model/119/3
class LossNetwork(torch.nn.Module):
    def __init__(self, vgg_model):
        super(LossNetwork, self).__init__()
        self.vgg_layers = vgg_model.features
        self.layer_name_mapping = {
            '3': "relu1_2",
            '8': "relu2_2",
            '15': "relu3_3",
            '22': "relu4_3"
        }
    
    def forward(self, x):
        output = {}
        for name, module in self.vgg_layers._modules.items():
            x = module(x)
            if name in self.layer_name_mapping:
                output[self.layer_name_mapping[name]] = x
        return LossOutput(**output)

# loss_net=LossNetwork(models.vgg19(pretrained=True))

def perp_loss(x,x_hat,p_loss):
    b=p_loss(x)-p_loss(x_hat)
    out=np.lilang.norm(b)
    return out
    
# class GramMatrix(nn.Module):
#     def forward(self, input):
#         b, c, h, w = input.size()
#         f = input.view(b, c, h * w) # b x c x(hxw)
#         # torch.bmm(batch1, batch2, out=None)
#         # batch1 : bxmxp, batch2 : bxpxn -> bxmxn
#         G = torch.bmm(f, f.transpose(1, 2)) 
#         # f: bxcx(hxw), f.transpose: bx(hxw)xc -> bxcxc
#         return G.div_(h * w)
    

In [15]:
np.array(digits).shape

(2000, 28, 28)

In [16]:
a=resize(np.array(digits))

NameError: name 'resize' is not defined

In [ ]:
a=torch.tensor(a)


In [ ]:
mnist_resized=aa
print(mnist_resized[0][0].shape)

In [ ]:
data_loader=DataLoader(mnist_d,batch_size=BATCH_SIZE,shuffle=True)
loss_l2 = torch.nn.MSELoss()
loss_l1=torch.nn.L1Loss()
writer = SummaryWriter()


def l2_l1_loss(true_img,out_img,l1,l2,net,ids):
    norm_weights=torch.square(net.embedding_content(ids))
    L1= (1/2)*l2(true_img,out_img)
    L2=(1/2)*l1(true_img,out_img)
    WDecay=WD*torch.sum(norm_weights)
    return L1+L2 + WDecay

col_content=[]
for j in range(CONTENT_DIM):
    col_content.append("x{}".format(j))

col_content.append("Epoch")

col_class=[]
for j in range(CLASS_DIM):
    col_class.append("x{}".format(j))
col_class.append("Epoch")


class_embeddings_time=pd.DataFrame(columns=col_class)
content_embedding_time=pd.DataFrame(columns=col_content)

j=0
optimizer = torch.optim.Adam(cur_model.parameters())

In [ ]:
for epoch in range(EPOCHS):
    total_loss=0
    i=0
    cur_model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        images,classes,ids=batch
        output=cur_model.forward(classes,ids)
        loss=l2_l1_loss(images.squeeze(),output.squeeze(),loss_l1,loss_l2,cur_model,ids)
#         loss=perp_loss(images,resized_for_vgg,loss_net)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        i+=1
    print("not crash")
    if epoch%10==0:
        conc_img = torch.cat((images[0],output[0][np.newaxis,:,:]),dim=2)
        writer.add_image("True image vs generated  in epoch {}".format(epoch)
                         ,conc_img)
        print("not crash")
        with torch.no_grad():
            activation = {}
            def get_activation(name):
                def hook(model, input, output):
                    activation[name] = output.detach()
                return hook
            print("not crash")

            model=cur_model
            num_contnet=1
            print("not crash")
            model.generator.net[10].register_forward_hook(get_activation('ext_conv1'))
            print("not crash")

            x = torch.tensor([2,1])
            y=torch.tensor([num_contnet,1])
            output = model(x,y)
            print("not crash")

            act = activation['ext_conv1'].squeeze()
            print("not crash")

            num_plot = 4
            fig, axarr = plt.subplots(4,4,figsize=(8,8))
            
            images=[]
            fig.suptitle("Activation layers , generate digit 2 with thick 4 content vector , epoch {}".format(epoch))
            for i,ax  in enumerate(axarr.flatten()):
                ax.imshow(act[0][i,:,:])
            print("not crash")

            cur_model.eval()
            print("not crash")

            pic=cur_model.forward(torch.tensor([2,1]),torch.tensor([1,1]))
            print("not crash")

            fig=plt.figure()
            fig.subplots_adjust(bottom = 0)
            fig.subplots_adjust(top = 1)
            fig.subplots_adjust(right = 1)
            fig.subplots_adjust(left = 0)
            a = torch.tensor(mnist_d.data[1][0]).squeeze()
            b = torch.tensor(mnist_d.data[num_contnet][0]).squeeze()
            c = pic[0]
            d=torch.cat((a,b,c),dim=1)
            print("not crash")

            fig = plt.figure(frameon=False, facecolor='white')
            ax = plt.Axes(fig, [0., 0., 1., 1.])
            ax.set_axis_off()
            fig.add_axes(ax)
            ax.imshow(d)
            plt.show()

        cur_model.save_weights("weights_trained\\trained_mnist_gen_loss_{}".format(str(total_loss/i)[2:6]))
        print("loss in epoch {}".format(epoch),total_loss/i)

#     if epoch%5==0:
#         class_embeddings=np.zeros((10,CLASS_DIM+1))
#         content_embeddings=np.zeros((100,CONTENT_DIM+1))
        
#         with torch.no_grad():
#             for i in range(10):
#                 class_embeddings[i,:CLASS_DIM]=cur_model.embedding_class(torch.tensor(i))
#             for i in range(100):
#                 content_embeddings[i,:CONTENT_DIM]=cur_model.embedding_content(torch.tensor(i))


        # class_embeddings[:,CLASS_DIM]=epoch
        # content_embeddings[:,CONTENT_DIM]=epoch
        # df = pd.DataFrame(class_embeddings,columns=col_class)
        # class_emb_df=df
        # content_emb_df_curr=pd.DataFrame(content_embeddings,columns=col_content)

        # content_embedding_time=content_embedding_time.append(content_emb_df_curr)
        # class_embeddings_time=class_embeddings_time.append(class_emb_df)
        #
        j+=1
        # writer.add_embedding(class_embeddings, metadata=np.arange(10),tag="Class embeddings epoch"+str(epoch),label_img=torch.tensor(num_pics))
        # writer.add_embedding(content_embeddings, metadata=np.arange(100),tag="Content embeddings epoch {}".format(epoch),label_img=torch.tensor(image[:100,:,:,:]))

    # writer.add_scalar("loss",total_loss/i,epoch)
# writer.flush()

In [ ]:
%tensorboard --logdir run


In [ ]:
with torch.no_grad():
    class_embeddings=np.zeros((10,CLASS_DIM))
    for i in range(10):
        class_embeddings[i,:]=cur_model.get_emb_class(torch.tensor(i))

    id_embeddings=np.zeros((2000,CONTENT_DIM))
    for id in range(2000):
        id_embeddings[id,:]=cur_model.get_emb_content(torch.tensor(id))


In [ ]:
writer.add_embedding(class_embeddings, metadata=np.arange(10),tag="Class embeddings1")
writer.add_embedding(id_embeddings,metadata=np.arange(2000),tag="Content embedding1",label_img=torch.tensor(image))

In [ ]:
import plotly.express as px
from  scipy.spatial import distance_matrix
import seaborn as sns
from sklearn.decomposition import PCA
curr_PCA=PCA(n_components=2)
curr_embeddings=curr_PCA.fit_transform(class_embeddings)
curr_data=pd.DataFrame(curr_embeddings,columns=["PC1","PC2"])
curr_data["class_num"]=np.arange(10)
display(curr_data)
fig=px.scatter(curr_data,hover_data=["class_num"],x="PC1",y="PC2")
fig.show()
print(curr_data.shape)

Does the class embeddings make sense?

In [ ]:
import plotly.express as px
import seaborn as sns
from  scipy.spatial import distance_matrix
col_epochs=(np.full(5,10))*(np.arange(0,5))
distance_transition=pd.DataFrame(columns=col_epochs)
for i in col_epochs:
    class_embeddings_curr=class_embeddings_time[class_embeddings_time["Epoch"]==i]
    dist_mat=distance_matrix(class_embeddings_curr,class_embeddings_curr)
    dist_mat=np.where(dist_mat==0,np.inf,dist_mat)
    distance_transition[i]=dist_mat.argmin(axis=1)

distance_transition.insert(0,"labels",np.arange(10))
plt.figure(figsize=((15,10)))
display(pd.DataFrame(np.array(distance_transition[20]).reshape(1,-1)))
pd.plotting.parallel_coordinates(distance_transition, 'labels',sort_labels=True,colormap="Paired")
plt.title("Class embedding nearest neighbor based on epochs with weight decay  {} ".format(WD))
plt.xlabel("Epochs")
plt.ylabel("Digits")
plt.yticks(np.arange(0,10))
# fig = px.parallel_coordinates(distance_transition,color="labels",title="Class embedding nearest neighbor based on epochs")
# fig.show()



In [ ]:
curr_PCA=PCA(n_components=3)
curr_embeddings=curr_PCA.fit_transform(id_embeddings)
curr_data=pd.DataFrame(curr_embeddings,columns=["PC1","PC2","PC3"])
curr_data["id"]=np.arange(2000)
display(curr_data)
fig=px.scatter(curr_data,hover_data=["id"],x="PC1",y="PC2",color="PC3")
fig.show()
print(curr_data.shape)

In [ ]:
cur_model.eval()
curr_pic=90
curr_num=3
with torch.no_grad():
    pic=cur_model.forward(torch.tensor([curr_num,1]),torch.tensor([curr_pic,1]))
    fig=plt.figure()
    fig.subplots_adjust(bottom = 0)
    fig.subplots_adjust(top = 1)
    fig.subplots_adjust(right = 1)
    fig.subplots_adjust(left = 0)
    a = torch.tensor(mnist_d.data[curr_pic][0]).squeeze()
    b = torch.tensor(mnist_d.data[curr_pic][0]).squeeze()
    c = pic[0]
    d=torch.cat((a,b,c),dim=1)

    fig = plt.figure(frameon=False, facecolor='white')
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(d)

In [ ]:

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model=cur_model
model.generator.net[10].register_forward_hook(get_activation('ext_conv1'))
x = torch.tensor([4,1])
y=torch.tensor([20,1])
output = model(x,y)
zz=(torch.tensor(mnist_d.data[7][0]).squeeze())
xx=torch.tensor(mnist_d.data[20][0]).squeeze()

with torch.no_grad():
    d=torch.cat((zz,xx,output[0]),dim=1)
    plt.imshow(d)
act = activation['ext_conv1'].squeeze()
num_plot = 4
print(act.shape)
fig, axarr = plt.subplots(4,4,figsize=(8,8))
images=[]
print(axarr.shape)
print(act.shape)
fig.suptitle("Activation layers , generate digit 5 with rotated content vector")
for i,ax  in enumerate(axarr.flatten()):
    ax.imshow(act[0][i,:,:])
